In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('./data/체육시설.csv')

In [5]:
df.isna().sum()

체육시설명    0
주소       0
위도       0
경도       0
dtype: int64

In [4]:
df.head()

,체육시설명,주소,위도,경도
0,서구실내게이트볼장,인천광역시 서구 원창로 124,37.516931,126.662868
1,원당게이트볼장,인천광역시 서구 원당동 850-2,37.597200,126.703004
2,마전게이트볼장,"인천광역시 서구 마전동 905-6, 905-7",37.615866,126.657588
3,당하게이트볼장,인천광역시 서구 당하동 1097-2,37.592040,126.674083
4,태정마을게이트볼장,인천광역시 서구 대곡동 490-9,37.621514,126.670253


In [6]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 시설 위도, 경도 배열
locations = df[['위도', '경도']].to_numpy()

# 위도, 경도를 라디안으로 변환
locations_rad = np.radians(locations)

# k-NN 모델 초기화 (3개 최근접 시설 찾기)
nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
nbrs.fit(locations_rad)

# 사용자 위치 (예: 위도, 경도)
user_location = (37.542996, 126.677156)  # 학원 위치
user_loc_rad = np.radians([user_location])

# 최근접 3개 시설 찾기
distances, indices = nbrs.kneighbors(user_loc_rad)

# 거리(km)로 변환 (각 시설마다)
distances_km = distances[0] * 6371  

# 찾은 3개 시설 정보 출력
for i, (idx, dist) in enumerate(zip(indices[0], distances_km), start=1):
    facility = df.iloc[idx]
    print(f"{i}번째 가까운 시설: {facility['체육시설명']}")
    print(f"주소: {facility['주소']}")
    print(f"거리: {dist:.2f} km\n")


1번째 가까운 시설: 연희배수지 인라인스케이트장
주소: 인천광역시 서구 서곶로314번길 10
거리: 0.34 km

2번째 가까운 시설: 국제대로하부족구장
주소: 인천광역시 서구 가정동 20
거리: 0.60 km

3번째 가까운 시설: 검암근린공원 국궁장
주소: 인천광역시 서구 서곶로 468-19
거리: 1.85 km



In [7]:
import folium

# 인천 중심 좌표 설정
incheon_center = [37.456255, 126.705206]

# 기본 지도 생성
m = folium.Map(location=incheon_center, zoom_start=11)

In [10]:
# 사용자 위치 마커 추가
folium.Marker(location=user_location, popup='내 위치', icon=folium.Icon(color='red')).add_to(m)

# 가장 가까운 3개 시설 마커 강조 표시 (색상 변경 등)
for idx in indices[0]:
    facility = df.iloc[idx]
    folium.Marker(location=[facility['위도'], facility['경도']],
                  popup=facility['체육시설명'],
                  icon=folium.Icon(color='blue')).add_to(m)

In [11]:
m